In [13]:
import os

# List the current directory
#print(os.listdir(.))

In [14]:
# Ask the user to select a digit in the range 0 to 8
k = int(input("Please select a digit in the range 0 to 8: "))

# Check if the digit is within the valid range
if 0 <= k <= 8:
    print(f"will be running in k{k}")
else:
    print("Invalid selection. Please select a digit between 0 and 8.")
    exit(1)

from pathlib import Path

wd = Path(f"/home/rfriedma/src/k{k}/ceph/build")
os.chdir(wd)
%env CEPH_JTEST_ROOT=/home/rfriedma/src/k{k}/ceph/build
!echo $CEPH_JTEST_ROOT > /tmp/jpath


will be running in k3
env: CEPH_JTEST_ROOT=/home/rfriedma/src/k3/ceph/build


In [15]:
!ls -l
!bash -c MGR=0 ls -l


total 9736
drwxr-xr-x.  2 rfriedma rfriedma   12288 Dec 19 08:11 bin
drwxr-xr-x.  6 rfriedma rfriedma      54 Dec 18 07:40 boost
-rw-r--r--.  1 rfriedma rfriedma 6430776 Dec 18 07:38 build.ninja
-rw-r--r--.  1 rfriedma rfriedma    4873 Dec 19 08:31 ceph.conf
-rw-r--r--.  1 rfriedma rfriedma   90217 Dec 18 07:38 CMakeCache.txt
drwxr-xr-x. 10 rfriedma rfriedma    4096 Dec 18 07:41 CMakeFiles
-rw-r--r--.  1 rfriedma rfriedma    2665 Dec 18 07:38 cmake_install.cmake
-rw-r--r--.  1 rfriedma rfriedma 3348856 Dec 18 07:38 compile_commands.json
-rw-r--r--.  1 rfriedma rfriedma     389 Aug  5 08:28 CTestTestfile.cmake
drwxr-xr-x.  7 rfriedma rfriedma      68 Dec 19 08:31 dev
drwxr-xr-x.  3 rfriedma rfriedma      78 Dec 18 07:38 doc
drwxr-xr-x.  3 rfriedma rfriedma      20 Aug  5 08:28 etc
drwxr-xr-x.  2 rfriedma rfriedma    4096 Dec 18 07:38 include
-rw-------.  1 rfriedma rfriedma     228 Dec 19 08:31 keyring
drwxr-xr-x.  3 rfriedma rfriedma    8192 Dec 19 08:11 lib
drwxr-xr-x.  4 rfriedma rfr

In [16]:
%%bash
scrtch=to_"`date +%d_%H%M`"
echo $scrtch

MDS=0 MGR=1 OSD=5 MON=1 ../src/vstart.sh -n  --without-dashboard --msgr2 -X --memstore -o "memstore_device_bytes=68435456" -o "osd_op_queue=wpq"
sleep 2
bin/ceph -s

bin/ceph tell osd.* config set debug_osd 20/20

bin/ceph config set global osd_pool_default_pg_autoscale_mode off
sleep 2

# disable rescheduling of the queue due to 'no-scrub' flags
#bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999

# initial set of global scrub scheduling parameters
bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0
bin/ceph tell osd.* config set osd_scrub_min_interval 10
bin/ceph tell osd.* config set osd_scrub_max_interval 2000
bin/ceph tell osd.* config set osd_deep_scrub_interval 600


#PL1 is of size 3

bin/ceph osd pool create pl1 8 8
#bin/ceph osd pool autoscale-status
sleep 1
pl1_num=`bin/ceph osd pool stats pl1 | sed -n -r -e 's/.*id[^0-9]*([0-9]+)$/\1/p'`
echo $pl1_num
bin/ceph osd pool set pl1 size 3
bin/ceph osd pool set pl1 min_size 3
bin/ceph osd pool set pl1 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl1 noscrub 1
bin/ceph osd pool set pl1 nodeep-scrub 1
sleep 2

bin/rados bench -p pl1 -t 1 1 write -b 4096 --max-objects 8  --no-cleanup; 
bin/rados bench -p pl1 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee

#PL2

bin/ceph osd pool create pl2 16 16
bin/ceph osd pool set pl2 size 3
bin/ceph osd pool set pl2 min_size 3
bin/ceph osd pool set pl2 pg_autoscale_mode off
bin/ceph osd pool stats
bin/ceph osd pool set pl2 noscrub 1
bin/ceph osd pool set pl2 nodeep-scrub 1
sleep 2

bin/rados bench -p pl2 -t 1 1 write -b 4096 --max-objects 64  --no-cleanup; 
bin/rados bench -p pl2 1 write -b 4096 --max-objects 128 --show-time --no-cleanup --run-name eeeee
sleep 1
bin/ceph tell osd.* config set debug_osd 20/20

# the set of PGs
bin/ceph pg dump
bin/ceph pg dump pgs_brief

to_19_1129


rm -f core* 


hostname o10
ip 172.21.64.10
port 40756


/mnt/nvme0/src/k3/ceph/build/bin/ceph-authtool --create-keyring --gen-key --name=mon. /mnt/nvme0/src/k3/ceph/build/keyring --cap mon 'allow *' 


creating /mnt/nvme0/src/k3/ceph/build/keyring


...

*** DEVELOPER MODE: setting PATH, PYTHONPATH and LD_LIBRARY_PATH ***
2024-12-19T11:29:58.666-0600 7f9f246006c0 -1 WARNING: all dangerous and experimental features are enabled.
2024-12-19T11:29:58.689-0600 7f9f246006c0 -1 WARNING: all dangerous and experimental features are enabled.


osd.0: {
    "success": ""
}
osd.1: {
    "success": ""
}
osd.2: {
    "success": ""
}
osd.3: {
    "success": ""
}
osd.4: {
    "success": ""
}


*** DEVELOPER MODE: setting PATH, PYTHONPATH and LD_LIBRARY_PATH ***
2024-12-19T11:29:58.979-0600 7fb3868006c0 -1 WARNING: all dangerous and experimental features are enabled.
2024-12-19T11:29:58.994-0600 7fb3852006c0 -1 WARNING: all dangerous and experimental features are enabled.


version 39
stamp 2024-12-19T11:29:58.236880-0600
last_osdmap_epoch 0
last_pg_scan 0
PG_STAT  OBJECTS  MISSING_ON_PRIMARY  DEGRADED  MISPLACED  UNFOUND  BYTES   OMAP_BYTES*  OMAP_KEYS*  LOG  LOG_DUPS  DISK_LOG  STATE         STATE_STAMP                      VERSION  REPORTED  UP       UP_PRIMARY  ACTING   ACTING_PRIMARY  LAST_SCRUB  SCRUB_STAMP                      LAST_DEEP_SCRUB  DEEP_SCRUB_STAMP                 SNAPTRIMQ_LEN  LAST_SCRUB_DURATION  SCRUB_SCHEDULING                                            OBJECTS_SCRUBBED  OBJECTS_TRIMMED
2.f            0                   0         0          0        0       0            0           0    0         0         0  active+clean  2024-12-19T11:29:51.332596-0600      0'0     22:25  [0,4,1]           0  [0,4,1]               0         0'0  2024-12-19T11:29:48.308569-0600              0'0  2024-12-19T11:29:48.308569-0600              0                    0  periodic scrub scheduled @ 2024-12-19T11:29:58.841287-0600                 0        

dumped all


In [17]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
jp=$CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    %env CEPH_JTEST_ROOT=$jp
fi
cd $jp


--------> /home/rfriedma/src/k3/ceph/build


In [ ]:
%%bash

file_path="/tmp/jpath"
if [ -f "$file_path" ]; then
        file_contents=$(cat "$file_path")
        echo "File contents read into variable."
else
        echo "File does not exist."
fi

In [ ]:
%%bash
nosd=5
for ((i=0;i<=$nosd;i++)); do
  echo "Query map for OSD $i"
  bin/ceph pg $pl1_num.$i query | jq '.scrubber' 
done


In [ ]:
%%bash

#cd $CEPH_JTEST_ROOT

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00_b4params.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01_b4params.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02_b4params.json


# set the scheduling parameters

bin/ceph osd pool set pl1 scrub_min_interval 1
bin/ceph osd pool set pl1 scrub_max_interval 2
bin/ceph osd pool set pl1 deep_scrub_interval 1000

#bin/ceph osd pool set pl2 scrub_min_interval 2
#bin/ceph osd pool set pl2 scrub_max_interval 4
#bin/ceph osd pool set pl2 deep_scrub_interval 10

sleep 3

bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_00.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_01.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_02.json
#bin/ceph tell osd.3 dump_scrubs --format=json-pretty > /tmp/ds_02.json





In [ ]:
%%bash

#%cd $CEPH_JTEST_ROOT

# common scrub configs
bin/ceph tell osd.* config set osd_blocked_scrub_grace_period 20
bin/ceph tell osd.* config set osd_stats_update_period_scrubbing 2
bin/ceph tell osd.* config set osd_stats_update_period_not_scrubbing 3
#bin/ceph tell osd.* config set osd_scrub_backoff_ratio 0.9999
#bin/ceph tell osd.* config set osd_scrub_interval_randomize_ratio 0.1
#bin/ceph tell osd.* config set osd_deep_scrub_randomize_ratio 0

#bin/ceph tell mgr.$(bin/ceph mgr services | jq -r .mgr) config set mgr_stats_period 2


In [ ]:
%%bash

# list the scrub queue
scrtch=to_"`date +'%H%M%S'`"
echo $scrtch
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json


In [ ]:
%%bash
# set scrub parameters to guarantee slow scrub
bin/ceph tell osd.* config set osd_scrub_sleep "3.0"
bin/ceph tell osd.* config set osd_max_scrubs 1
bin/ceph tell osd.* config set osd_scrub_chunk_max 5
bin/ceph tell osd.* config set osd_shallow_scrub_chunk_max 5


In [ ]:
%%bash

# set higher urgency to one of the PGs
bin/ceph tell $pl1_num.7 scrub
bin/ceph tell $pl1_num.6 schedule-deep-scrub
sleep 1
bin/ceph pg dump pgs


In [ ]:
%%bash

scrtch=to_"`date +'%H%M%S'`"
echo $scrtch

# list the scrub queue
bin/ceph tell osd.0 dump_scrubs --format=json-pretty > /tmp/ds_0$scrtch.json
bin/ceph tell osd.1 dump_scrubs --format=json-pretty > /tmp/ds_1$scrtch.json
bin/ceph tell osd.2 dump_scrubs --format=json-pretty > /tmp/ds_2$scrtch.json
bin/ceph tell osd.0 dump_scrubs --format=json-pretty
bin/ceph tell osd.1 dump_scrubs --format=json-pretty
bin/ceph tell osd.2 dump_scrubs --format=json-pretty


In [ ]:
raise SystemExit("Stop here")

In [ ]:
%%bash

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)]| sort_by(.overdue,.pgid)' |  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)|  { pgid, overdue, sched_time } ]' |  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true)|  { pgid, overdue, sched_time } ]' |  head -20

#cat ds_01.json | jq -s '[.[]|.[]|select(.eligible==false) | .overdue as $ov | . += { "ov":$ov } | [.] ] '|  head -20

#cat ds_01.json | jq -s '[.[]|.[]|select(.eligible==false) | .overdue as $ov | . += { "ov":$ov|not } | [.] ] '|  head -20

cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | . += { "ov":$ov|not } ]| sort_by(.ov,.sched_time,.pgid) | [.]  '|  head -20

#cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | .level as $lvl | . += { "ov":$ov|not, "lvl":$lvl } ]| sort_by(.ov,.sched_time,.pgid,.lvl) | [.]  '|  head -20

cat /tmp/ds_01.json | jq -s '[.[]|.[]|select(.eligible==true) | .overdue as $ov | .level as $lvl |
 . += { "ov":$ov|not, "lvl":$lvl } ]| sort_by(.ov,.sched_time,.pgid,.lvl) | [.]  '|  head -20

# use 'eligible' as just one more sort criteria
echo "========================== --- "
cat /tmp/ds_01.json | jq -s '[.[]|.[]| .eligible as $ripe | .overdue as $ov | .level as $lvl |
 . += { "not_ripe":$ripe|not, "not_ov":$ov|not, "lvl":$lvl } ] | 
 sort_by(.not_ripe, .not_ov,.sched_time,.pgid,.lvl) | [.]  '|  head -100

# now - make the sort an irregular one: if comparing the the targets of one PG, level tramps sched time



# Termination


In [ ]:
%%bash
echo '-------->' $CEPH_JTEST_ROOT
cd $CEPH_JTEST_ROOT
if [ -z $CEPH_JTEST_ROOT ]; then
    echo "CEPH_JTEST_ROOT is not set"
    [[ -f /tmp/jpath ]] && jp=`cat /tmp/jpath` || jp='.'
    echo '-------->' $jp
    cd $jp
    %env CEPH_JTEST_ROOT=$jp
fi

pwd

../src/stop.sh
sleep 4
../src/stop.sh
